# Εργασία 2 ΤΕΔεδομένων

Κωνσταντίνος Σκορδούλης 1115201600155

Ευστράτιος Ζωγραφάκης 1115201600049

In [1]:
import pandas as pd
import numpy as np

In [2]:
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score

In [3]:
#import statistics #for standard deviation
from statistics import stdev

Load our dataframe

In [4]:
#train_name = './train_set_testing.tsv' #HERE
train_name = './train_set.tsv'

#test_name = './test_set_testing.tsv'#HERE
test_name = './test_set.tsv'

#y_test_name = './Y_test_testing.tsv' #HERE
y_test_name = './Y_test.tsv'


Train_df = pd.read_csv(train_name,sep='\t', usecols=['Id','Title','Content','Category'])
#Train_df.index = Train_df['Id']

Test_df = pd.read_csv(test_name,sep='\t', usecols=['Id','Title','Content'])
#Test_df.index = Test_df['Id']

Y_test =  pd.read_csv(y_test_name,sep='\t')
#Y_test.index = Test_df['Id']

#Train_df

Encoding our labels with <b>LabelEncoder</b>

In [5]:
from sklearn.preprocessing import LabelEncoder

In [6]:
lb = LabelEncoder()
lb.fit(Train_df['Category'])

labels_test = lb.transform(Y_test['Category'])
labels_train = lb.transform(Train_df['Category'])

# Document-words

Create <b>bag of Words</b> (μόνο για το Content)

In [7]:
from sklearn.feature_extraction.text import CountVectorizer

In [8]:
bow_vectorizer = CountVectorizer(max_df=1.0, min_df=1, max_features=1000,
stop_words='english')

counts_train = bow_vectorizer.fit_transform(Train_df.Content)
counts_test = bow_vectorizer.transform(Test_df.Content)

Transfrom count to tfidf --> <b>TFidf_Transformer()</b> ---> maybe TfidfVectorizer()

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [10]:
tfidf_vectorizer = TfidfVectorizer(max_df=1.0, min_df=1, max_features=1000,
stop_words='english')

tfidf_train = tfidf_vectorizer.fit_transform(Train_df.Content)
tfidf_test = tfidf_vectorizer.transform(Test_df.Content)

# KNN

In [11]:
from collections import Counter

In [12]:
from scipy.sparse import issparse

In [13]:
def knn_predict1(X_train,Y_train,X_test,k,knn_fold):

    
    dist = [] # a list of list (dist,index_train)
    neighbors = []
    
    
    for i in range(X_train.shape[0]): #for each train data point
        
        x1 = X_test - X_train[i,:]        
        x2 = np.square([x1])
        x3 = np.sqrt(np.sum(x2))
        dist.append([x3,i])
            
    #sort distances(ascending)
    dist = sorted(dist)
    
    #Find k-nearest neighbors
    for i in range(k):
        index = dist[i][1]
        neighbors.append(Y_train[index])
    
    #Now for the Majority Vote(use Countw)
    c1 = Counter(neighbors)    
    
    if knn_fold == False:
        result = c1.most_common(1) # returns [(word,frequency)]
        return result[0][0] #we only want the word  
    else:
        result = c1.most_common(k)
        return result # we want this list of tupples

In [14]:
def knn(X_train,Y_train,X_test,k,knn_fold):
    
    result = []
    
    result1 = [] #in case of knn_fold = true
    for i in range(X_test.shape[0]):
 
        x = knn_predict1(X_train,Y_train,X_test[i,:],k,knn_fold)
    
        if( knn_fold == True):
            result1.append(x[0][0])            
        result.append(x)
    
    if( knn_fold == True):
        return [result1,result] #result1 (list) contains the predicted labels
        #while result (list of lists of tupples) which is usefull for computing the probabilities (roc_auc_score)
    
    
    #else just return the labels
    return result #contains label for each X_test  (or a list of lists, containing tupples (word,frequency) )

We choose <b>k</b> as the <b>sqrt( len( X_train ) )</b>

In [15]:
k = int(np.sqrt(len(Train_df)))

if( k % 2 == 0): #we want odd number
    k = k+1

In [16]:
scoring_labels = ['precision_macro','recall_macro','f1_macro','balanced_accuracy','roc_auc_ovo']

Create our very own cross_val_score(), using the functions below

In [17]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, balanced_accuracy_score, roc_auc_score

In [18]:
def knn_10fold(X_train,Y_train,k,label_encoder):
#def knn_10fold(X_train,Y_train,k):

    label_order = ['business','entertainment','politics','sport','tech']

    precision_list = []
    recall_list = []
    f1_list = []
    accuracy_list = []
    roc_auc_list = []        
    
    for i in range(10):
        
        X_train1,X_test1,Y_train1,Y_test1 = train_test_split(X_train,Y_train, test_size=0.1, stratify=Y_train)
        
        Y_train1 = Y_train1.reset_index(drop=True) #reset indices    
        Y_pred, list_coll = knn(X_train1,Y_train1,X_test1,k,True)
        
        
        #Now get the probability matrix for
        Y_pred_proba = Compute_proba(list_coll,label_order,k)
        
        
        Y_pred = label_encoder.transform(Y_pred) #encode the predicted labels
        Y_test2 = label_encoder.transform(Y_test1) #encode the true test label
        
        precision_macro = precision_score(Y_test2, Y_pred, average='macro')
        recall_macro = recall_score(Y_test2,Y_pred, average='macro')
        f1_macro = f1_score(Y_test2, Y_pred, average='macro')
        accuracy_b = balanced_accuracy_score(Y_test2, Y_pred)
        
        
        #roc_auc = roc_auc_score(Y_test2, Y_pred, average='macro', multi_class='ovo')
        roc_auc = roc_auc_score(Y_test2,Y_pred_proba, average='macro', multi_class='ovo')
        
        precision_list.append(precision_macro)
        recall_list.append(recall_macro)
        f1_list.append(f1_macro)
        accuracy_list.append(accuracy_b)
        roc_auc_list.append(roc_auc)

    
    return [precision_list,recall_list,f1_list,accuracy_list,roc_auc_list]



In [19]:
def Compute_proba(list1,label_order,k):
    
   #list1 is a list of list of tupples (word,frequency_int)
    
    final_list = [] #our final array for the roc_auc_score

    for l in list1: #for each test point
    
        list2 = [] #probabilities of this test point
    
        dict1={} #contains the probabilities of each label
        for x in l:
            dict1[x[0]] = x[1] / k
        #end of loop
        
        for x in label_order: # ['business','entertainment','politics','sport','tech']
            
            if x not in dict1.keys():
                list2.append(0)
            else:
                list2.append(dict1[x])
                
        #end of loop
        
        final_list.append(list2)

    return np.array(final_list)  

In [20]:
def print_kfold_scores(scores):
    #scores is a list of scores
    
    print("Precision_macro is (mean : %f) and (standard deviation: %f)" % ( np.mean(scores[0]), stdev(scores[0]) ) )
    print("Recall_macro is (mean : %f) and (standard deviation: %f)" % ( np.mean(scores[1]), stdev(scores[1]) ) )
    print("f1_macro is (mean : %f) and (standard deviation: %f)" % ( np.mean(scores[2]), stdev(scores[2]) ) )
    
    print("balanced_accuracy is (mean : %f) and (standard deviation: %f)" % ( np.mean(scores[3]), stdev(scores[3]) ) )
    print("Roc_auc_ovo is (mean : %f) and (standard deviation: %f)" % ( np.mean(scores[4]), stdev(scores[4]) ) )   

# KNN count

knn count prediction and model accuracy

In [21]:
count_pred_knn = knn(counts_train.toarray(),Train_df['Category'],counts_test.toarray(),k,False)

In [22]:
count_pred_knn1 = lb.transform(count_pred_knn)
accuracy_score(labels_test, count_pred_knn1 )

0.5528089887640449

Now for the 10-fold cross_val_score()

In [23]:
score_list = knn_10fold(counts_train.toarray(),Train_df['Category'],k,lb)
print_kfold_scores(score_list)

Precision_macro is (mean : 0.764687) and (standard deviation: 0.024675)
Recall_macro is (mean : 0.559045) and (standard deviation: 0.025571)
f1_macro is (mean : 0.564736) and (standard deviation: 0.030575)
balanced_accuracy is (mean : 0.559045) and (standard deviation: 0.025571)
Roc_auc_ovo is (mean : 0.951276) and (standard deviation: 0.011237)


# KNN tfidf

In [24]:
tfidf_pred_knn = knn(tfidf_train.toarray(),Train_df['Category'],tfidf_test.toarray(),k,False)

In [25]:
tfidf_pred_knn1 = lb.transform(tfidf_pred_knn)
accuracy_score(labels_test, tfidf_pred_knn1 )

0.9370786516853933

Now for the 10-fold cross_val_score()

In [26]:
score_list = knn_10fold(tfidf_train.toarray(),Train_df['Category'],k,lb)
print_kfold_scores(score_list)

Precision_macro is (mean : 0.937173) and (standard deviation: 0.026319)
Recall_macro is (mean : 0.934403) and (standard deviation: 0.026672)
f1_macro is (mean : 0.934897) and (standard deviation: 0.026679)
balanced_accuracy is (mean : 0.934403) and (standard deviation: 0.026672)
Roc_auc_ovo is (mean : 0.996058) and (standard deviation: 0.002440)


# Naive Bayes

Όπως με κάθε classifier, θα χρειαστούμε να:

1) να βρούμε τις κατάλληλες hyperparameters (ανάλογα με το training dataset-> counts or tfidf) με τη βοήθεια του GridSearchCV

2) Να κάνουμε train/fit to μοντέλο μας (με χρήση του training_data)

3) Να κάνουμε predict (με χρήση του testing_data)

4) Scoring

    a) Accuracy του predict
   
    b) 10-fold crοss validation -> cross_val_score() --> όλα όσα ζητάει (precision κλπ). 
    Θέλουμε μέσο όρο και τυπική απόκλιση για κάθε score

In [27]:
from sklearn.naive_bayes import MultinomialNB

# Naive Bayes counts

Compute hyperparameters

In [28]:
#alphas = [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0]
#alphas = [0.1,0.01,0.001,0.0001,0.0001,0.000001] #0.0010
#alphas = [0.0003, 0.0008] #0.0008
#alphas = [0.0009, 0.00095] #0.0009
#alphas = [0.00092,0.00094] #0.00092
#alphas = [0.00091,0.00092] #0.00091
#alphas = [0.000905, 0.00091] #0.00905
#alphas = [0.000902,0.000904] #0.000902

alphas = [0.000902] # ===> OK

#pgrid = {'alpha': alphas, 'fit_prior': [True,False],'class_prior': [None, [0.1,0.9], [0.2,0.8]] }
pgrid = {'alpha': alphas, 'fit_prior': [True,False],'class_prior': [None] }
#Number of priors must match number of classes -> example: [0.1,0.1,0.1,0.1,0.6]

In [29]:
#mnb = MultinomialNB()

grid = GridSearchCV(estimator=MultinomialNB(), param_grid=pgrid, scoring=scoring_labels, refit='balanced_accuracy', cv=10)
grid.fit(counts_train,labels_train)

mnb = grid.best_estimator_ #returns the optimized model (in regards to the parameters)
grid.best_params_

{'alpha': 0.000902, 'class_prior': None, 'fit_prior': True}

Train/Fit our model

In [30]:
mnb.fit(counts_train,labels_train)

MultinomialNB(alpha=0.000902, class_prior=None, fit_prior=True)

Model.predict() and <b>accuracy of our prediction</b>

In [31]:
mnb_counts_pred = mnb.predict(counts_test)

In [32]:
accuracy_score(labels_test, mnb_counts_pred)

0.946067415730337

Cross_val_scores -> cross_validate()

In [33]:
precision_scores = cross_val_score(mnb, counts_train, labels_train, cv=10, scoring='precision_macro')
recall_scores = cross_val_score(mnb, counts_train, labels_train, cv=10, scoring='recall_macro')
f1_scores = cross_val_score(mnb, counts_train, labels_train, cv=10, scoring='f1_macro')
kfold_accuracy_scores = cross_val_score(mnb, counts_train, labels_train, cv=10, scoring='balanced_accuracy')
ROC_scores = cross_val_score(mnb, counts_train, labels_train, cv=10, scoring='roc_auc_ovo')

score_list = [precision_scores,recall_scores,f1_scores,kfold_accuracy_scores,ROC_scores]

In [34]:
print_kfold_scores(score_list)

Precision_macro is (mean : 0.969957) and (standard deviation: 0.010605)
Recall_macro is (mean : 0.968899) and (standard deviation: 0.011760)
f1_macro is (mean : 0.968772) and (standard deviation: 0.011755)
balanced_accuracy is (mean : 0.968899) and (standard deviation: 0.011760)
Roc_auc_ovo is (mean : 0.996706) and (standard deviation: 0.003415)


# Naive Bayes tfidf

Compute hyperparameters

In [35]:
#alphas = [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0]
#alphas = [0.1,0.01,0.001,0.0001,0.0001,0.000001] # 0.001
#alphas = [0.0005,0.0008] #0.0008
#alphas = [0.0008,0.00085] #0.0008

alphas = [0.0008] # ===> OK

#pgrid = {'alpha': alphas, 'fit_prior': [True,False],'class_prior': [None, [0.1,0.9], [0.2,0.8]] }
pgrid = {'alpha': alphas, 'fit_prior': [True,False],'class_prior': [None] }
#Number of priors must match number of classes -> example: [0.1,0.1,0.1,0.1,0.6]

In [36]:
#mnb = MultinomialNB()

grid = GridSearchCV(estimator=MultinomialNB(), param_grid=pgrid, scoring=scoring_labels, refit='balanced_accuracy', cv=10)
grid.fit(tfidf_train,labels_train)

mnb = grid.best_estimator_ #returns the optimized model (in regards to the parameters)
grid.best_params_

{'alpha': 0.0008, 'class_prior': None, 'fit_prior': True}

Train/fit our model

In [37]:
mnb.fit(tfidf_train,labels_train)

MultinomialNB(alpha=0.0008, class_prior=None, fit_prior=True)

Model.predict() and <b>accuracy of our prediction</b>

In [38]:
mnb_tfidf_pred = mnb.predict(tfidf_test)

In [39]:
accuracy_score(labels_test, mnb_tfidf_pred)

0.9483146067415731

Cross_val_scores -> cross_validate()

In [40]:
precision_scores = cross_val_score(mnb, tfidf_train, labels_train, cv=10, scoring='precision_macro')
recall_scores = cross_val_score(mnb, tfidf_train, labels_train, cv=10, scoring='recall_macro')
f1_scores = cross_val_score(mnb, tfidf_train, labels_train, cv=10, scoring='f1_macro')
kfold_accuracy_scores = cross_val_score(mnb, tfidf_train, labels_train, cv=10, scoring='balanced_accuracy')
ROC_scores = cross_val_score(mnb, tfidf_train, labels_train, cv=10, scoring='roc_auc_ovo')

score_list = [precision_scores,recall_scores,f1_scores,kfold_accuracy_scores,ROC_scores]

In [41]:
print_kfold_scores(score_list)

Precision_macro is (mean : 0.971517) and (standard deviation: 0.009526)
Recall_macro is (mean : 0.970354) and (standard deviation: 0.010638)
f1_macro is (mean : 0.970426) and (standard deviation: 0.010496)
balanced_accuracy is (mean : 0.970354) and (standard deviation: 0.010638)
Roc_auc_ovo is (mean : 0.998438) and (standard deviation: 0.001265)


#  Random Forests

In [42]:
from sklearn.ensemble import RandomForestClassifier

# Random Forests count

Compute HyperParameters

In [43]:
#Use Gini to minimize miss-classification

#max_depth = [10,20,30,40,50,60,70,80,90,100]
#max_depth = [10,50,100]
#max_depth=[5,10] # 10
#max_depth=[15,30] # 30
#max_depth=[35,60] #35
#max_depth=[32,34] #34

max_depth=[34] # ====> OK



#max_features = [1,2,3,4,5,6,7,8,9,10]
#max_features=[2,5,10] #10
#max_features=[15,30] # 15
#max_features=[11,12,13,14,15] # 11

max_features = [11] # =====> OK




#min_samples_leaf = [1,2,3,4,5,6,7,8,9,10]
#min_samples_leaf=[2,5,10] #5
#min_samples_leaf=[3,4,5] # 3

min_samples_leaf=[3] # ==> OK


#min_samples_split = [2,4,6,8,10,12,14,16]
#min_samples_split=[2,5,10] #5
#min_samples_split=[3,4,5] # 5
#min_samples_split=[5,6,7,8,9] # 9

min_samples_split=[9] #===> OK


#n_estimators = [10,50,100,150,200,250]
#n_estimators = [5,10] # 10
#n_estimators =[15,30] # 30
#n_estimators =[35,60] #60
#n_estimators = [100,200] #200
#n_estimators = [400,800] #800
#n_estimators = [1000,2500] #1000
#n_estimators = [900,1500] #1500
#n_estimators = [1600,2000] #1600
#n_estimators = [1530,1570] # 1570
#n_estimators = [1580,1590] #1590
#n_estimators =[1593,1597] #1593
#n_estimators = [1591,1592] #1591

n_estimators = [1591] # ====> OK


class_weight = ['balanced']

In [44]:
pgrid = {'bootstrap':[True], 'max_depth':max_depth, 'max_features':max_features, 'min_samples_leaf':min_samples_leaf, 
         'min_samples_split': min_samples_split, 'n_estimators':n_estimators,
        'class_weight':class_weight}

In [45]:
grid = GridSearchCV(estimator=RandomForestClassifier(), param_grid=pgrid, scoring=scoring_labels, refit='balanced_accuracy', cv=10)
grid.fit(counts_train,labels_train)

rfc = grid.best_estimator_ #returns the optimized model (in regards to the parameters)
grid.best_params_

{'bootstrap': True,
 'class_weight': 'balanced',
 'max_depth': 34,
 'max_features': 11,
 'min_samples_leaf': 3,
 'min_samples_split': 9,
 'n_estimators': 1591}

Train/Fit our model

In [46]:
rfc.fit(counts_train,labels_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight='balanced',
                       criterion='gini', max_depth=34, max_features=11,
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=3, min_samples_split=9,
                       min_weight_fraction_leaf=0.0, n_estimators=1591,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

Model.predict() and <b>accuracy of our prediction</b>

In [47]:
rfc_counts_pred = rfc.predict(counts_test)

In [48]:
accuracy_score(labels_test, rfc_counts_pred)

0.952808988764045

Cross_val_scores -> cross_validate()

In [49]:
precision_scores = cross_val_score(rfc, counts_train, labels_train, cv=10, scoring='precision_macro')
recall_scores = cross_val_score(rfc, counts_train, labels_train, cv=10, scoring='recall_macro')
f1_scores = cross_val_score(rfc, counts_train, labels_train, cv=10, scoring='f1_macro')
kfold_accuracy_scores = cross_val_score(rfc, counts_train, labels_train, cv=10, scoring='balanced_accuracy')
ROC_scores = cross_val_score(rfc, counts_train, labels_train, cv=10, scoring='roc_auc_ovo')

score_list = [precision_scores,recall_scores,f1_scores,kfold_accuracy_scores,ROC_scores]

In [50]:
print_kfold_scores(score_list)

Precision_macro is (mean : 0.964296) and (standard deviation: 0.013088)
Recall_macro is (mean : 0.962906) and (standard deviation: 0.012719)
f1_macro is (mean : 0.966497) and (standard deviation: 0.012189)
balanced_accuracy is (mean : 0.965819) and (standard deviation: 0.013712)
Roc_auc_ovo is (mean : 0.997928) and (standard deviation: 0.001271)


# Random Forests tfidf

Compute HyperParameters

In [51]:
#Use Gini to minimize miss-classification

#max_depth = [10,20,30,40,50,60,70,80,90,100]
#max_depth = [10,50,100]
#max_depth=[5,10]
#max_depth=[30,34] 


max_depth=[34] # ====> OK




#max_features = [1,2,3,4,5,6,7,8,9,10]
#max_features=[2,5,10]
#max_features = [9,11] 

max_features = [11] # =====> OK




#min_samples_leaf = [1,2,3,4,5,6,7,8,9,10]
#min_samples_leaf=[2,5,10]
#min_samples_leaf=[3,5] 

min_samples_leaf=[3] # ==> OK



#min_samples_split = [2,4,6,8,10,12,14,16]
#min_samples_split=[2,5,10]
#min_samples_split=[9,11] 

min_samples_split=[9] #===> OK



#n_estimators = [10,50,100,150,200,250]
#n_estimators = [5,10]
#n_estimators = [1591,1600] 

n_estimators = [1591] # ====> OK



class_weight = ['balanced']

In [52]:
pgrid = {'bootstrap':[True], 'max_depth':max_depth, 'max_features':max_features, 'min_samples_leaf':min_samples_leaf, 
         'min_samples_split': min_samples_split, 'n_estimators':n_estimators,
        'class_weight':class_weight}

In [53]:
grid = GridSearchCV(estimator=RandomForestClassifier(), param_grid=pgrid, scoring=scoring_labels, refit='balanced_accuracy', cv=10)
grid.fit(tfidf_train,labels_train)

rfc = grid.best_estimator_ #returns the optimized model (in regards to the parameters)
grid.best_params_

{'bootstrap': True,
 'class_weight': 'balanced',
 'max_depth': 34,
 'max_features': 11,
 'min_samples_leaf': 3,
 'min_samples_split': 9,
 'n_estimators': 1591}

Train/Fit our model

In [54]:
rfc.fit(tfidf_train,labels_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight='balanced',
                       criterion='gini', max_depth=34, max_features=11,
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=3, min_samples_split=9,
                       min_weight_fraction_leaf=0.0, n_estimators=1591,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

Model.predict() and <b>accuracy of our prediction</b>

In [55]:
rfc_tfidf_pred = rfc.predict(tfidf_test)

In [56]:
accuracy_score(labels_test, rfc_tfidf_pred)

0.952808988764045

Cross_val_scores -> cross_validate()

In [57]:
precision_scores = cross_val_score(rfc, tfidf_train, labels_train, cv=10, scoring='precision_macro')
recall_scores = cross_val_score(rfc, tfidf_train, labels_train, cv=10, scoring='recall_macro')
f1_scores = cross_val_score(rfc, tfidf_train, labels_train, cv=10, scoring='f1_macro')
kfold_accuracy_scores = cross_val_score(rfc, tfidf_train, labels_train, cv=10, scoring='balanced_accuracy')
ROC_scores = cross_val_score(rfc, tfidf_train, labels_train, cv=10, scoring='roc_auc_ovo')

score_list = [precision_scores,recall_scores,f1_scores,kfold_accuracy_scores,ROC_scores]

In [58]:
print_kfold_scores(score_list)

Precision_macro is (mean : 0.963017) and (standard deviation: 0.011765)
Recall_macro is (mean : 0.959470) and (standard deviation: 0.012975)
f1_macro is (mean : 0.958873) and (standard deviation: 0.013810)
balanced_accuracy is (mean : 0.957692) and (standard deviation: 0.013760)
Roc_auc_ovo is (mean : 0.998033) and (standard deviation: 0.001108)


# Support Vector Machine

In [59]:
from sklearn.svm import SVC

# SVC counts

Compute HyperParameters

In [60]:
#C = [1.0,5.0,10.0]
#C=[1.0,5.0] #5.0
#C=[10.0,100.0] #10.0
#C=[10.0,50.0] #10.0
#C= [10.0,12.0] # 10.0
#C= [7.0,10.0] #7.0
#C = [6.0,7.0] #7.0
#C = [7.5,9.0] #7.5
#C = [7.0,7.1,7.2,7.3,7.4,7.5] #7.0

C = [7.0] # ==> OK



#kernel = ['rbf','linear'] #rbf
kernel = ['rbf'] # ===> OK

#gamma = ['scale','auto'] #auto
gamma = ['auto'] # ===> OK


#######################
probability = [True]

class_weight = ['balanced']

decision_function_shape=['ovo'] #or 'ovr'


In [61]:
pgrid = {'kernel':kernel, 'C':C, 'gamma':gamma, 'decision_function_shape':decision_function_shape,
         'class_weight':class_weight, 'probability':probability}

#pgrid = {'kernel':kernel, 'C':C, 'gamma':gamma, 'probability':probability}

In [62]:
#grid = GridSearchCV(estimator=SVC(), param_grid=pgrid, scoring=scoring_labels, refit='balanced_accuracy', cv=10)
grid = GridSearchCV(estimator=SVC(), param_grid=pgrid, scoring=scoring_labels, refit='balanced_accuracy')

grid.fit(counts_train,labels_train)

svc = grid.best_estimator_ #returns the optimized model (in regards to the parameters)
grid.best_params_

{'C': 7.0,
 'class_weight': 'balanced',
 'decision_function_shape': 'ovo',
 'gamma': 'auto',
 'kernel': 'rbf',
 'probability': True}

Train/Fit our model

In [63]:
svc.fit(counts_train,labels_train)

SVC(C=7.0, break_ties=False, cache_size=200, class_weight='balanced', coef0=0.0,
    decision_function_shape='ovo', degree=3, gamma='auto', kernel='rbf',
    max_iter=-1, probability=True, random_state=None, shrinking=True, tol=0.001,
    verbose=False)

Model.predict() and <b>accuracy of our prediction</b>

In [64]:
svc_counts_pred = svc.predict(counts_test)

In [65]:
accuracy_score(labels_test, svc_counts_pred)

0.9438202247191011

Cross_val_scores -> cross_validate()

In [66]:
precision_scores = cross_val_score(svc, counts_train, labels_train, cv=10, scoring='precision_macro')
recall_scores = cross_val_score(svc, counts_train, labels_train, cv=10, scoring='recall_macro')
f1_scores = cross_val_score(svc, counts_train, labels_train, cv=10, scoring='f1_macro')
kfold_accuracy_scores = cross_val_score(svc, counts_train, labels_train, cv=10, scoring='balanced_accuracy')
ROC_scores = cross_val_score(svc, counts_train, labels_train, cv=10, scoring='roc_auc_ovo')

score_list = [precision_scores,recall_scores,f1_scores,kfold_accuracy_scores,ROC_scores]

In [67]:
print_kfold_scores(score_list)

Precision_macro is (mean : 0.962982) and (standard deviation: 0.010535)
Recall_macro is (mean : 0.962742) and (standard deviation: 0.010950)
f1_macro is (mean : 0.962460) and (standard deviation: 0.010719)
balanced_accuracy is (mean : 0.962742) and (standard deviation: 0.010950)
Roc_auc_ovo is (mean : 0.997473) and (standard deviation: 0.001481)


# SVC tfidf

We change from precision_macro to precision_micro, because there was slight class imbalance (even with stratified train_test_split). ===> We receive this error:

UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))

In [100]:
scoring_labels1 = ['precision_micro','recall_macro','f1_macro','balanced_accuracy','roc_auc_ovo']

In [110]:
#C = [1.0,5.0,10.0] # ==> 1.0
#C=[0.1,0.9] #0.9
#C=[0.9,1.0] #0.9

C = [0.9] # ===> OK


#kernel = ['rbf','linear']
kernel = ['rbf'] # ===> OK

#gamma = ['scale','auto']
gamma = ['scale'] # ===> OK


#######################
probability = [True]

class_weight = ['balanced']

decision_function_shape=['ovo'] #or 'ovr'

In [111]:
pgrid = {'kernel':kernel, 'C':C, 'gamma':gamma, 'decision_function_shape':decision_function_shape,
         'class_weight':class_weight, 'probability':probability}

#pgrid = {'kernel':kernel, 'C':C, 'gamma':gamma, 'probability':probability}

In [112]:
grid = GridSearchCV(estimator=SVC(), param_grid=pgrid, scoring=scoring_labels1, refit='balanced_accuracy', cv=10)
#grid = GridSearchCV(estimator=SVC(), param_grid=pgrid, scoring=scoring_labels, refit='balanced_accuracy')

grid.fit(tfidf_train,labels_train)

svc = grid.best_estimator_ #returns the optimized model (in regards to the parameters)
grid.best_params_

{'C': 0.9,
 'class_weight': 'balanced',
 'decision_function_shape': 'ovo',
 'gamma': 'scale',
 'kernel': 'rbf',
 'probability': True}

Train/Fit our model

In [113]:
svc.fit(tfidf_train,labels_train)

SVC(C=0.9, break_ties=False, cache_size=200, class_weight='balanced', coef0=0.0,
    decision_function_shape='ovo', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=True, random_state=None, shrinking=True, tol=0.001,
    verbose=False)

Model.predict() and <b>accuracy of our prediction</b>

In [114]:
svc_tfidf_pred = svc.predict(tfidf_test)

In [115]:
accuracy_score(labels_test, svc_tfidf_pred)

0.952808988764045

Cross_val_scores -> cross_validate()

In [116]:
#precision_scores = cross_val_score(svc, tfidf_train, labels_train, cv=10, scoring='precision_macro')
precision_scores = cross_val_score(svc, tfidf_train, labels_train, cv=10, scoring='precision_micro')


recall_scores = cross_val_score(svc, tfidf_train, labels_train, cv=10, scoring='recall_macro')
f1_scores = cross_val_score(svc, tfidf_train, labels_train, cv=10, scoring='f1_macro')
kfold_accuracy_scores = cross_val_score(svc, tfidf_train, labels_train, cv=10, scoring='balanced_accuracy')
ROC_scores = cross_val_score(svc, tfidf_train, labels_train, cv=10, scoring='roc_auc_ovo')

score_list = [precision_scores,recall_scores,f1_scores,kfold_accuracy_scores,ROC_scores]

In [117]:
#print_kfold_scores(score_list)

In [118]:
    print("Precision_micro is (mean : %f) and (standard deviation: %f)" % ( np.mean(score_list[0]), stdev(score_list[0]) ) )
    print("Recall_macro is (mean : %f) and (standard deviation: %f)" % ( np.mean(score_list[1]), stdev(score_list[1]) ) )
    print("f1_macro is (mean : %f) and (standard deviation: %f)" % ( np.mean(score_list[2]), stdev(score_list[2]) ) )
    
    print("balanced_accuracy is (mean : %f) and (standard deviation: %f)" % ( np.mean(score_list[3]), stdev(score_list[3]) ) )
    print("Roc_auc_ovo is (mean : %f) and (standard deviation: %f)" % ( np.mean(score_list[4]), stdev(score_list[4]) ) )

Precision_micro is (mean : 0.976404) and (standard deviation: 0.010855)
Recall_macro is (mean : 0.976140) and (standard deviation: 0.010580)
f1_macro is (mean : 0.976168) and (standard deviation: 0.010681)
balanced_accuracy is (mean : 0.976140) and (standard deviation: 0.010580)
Roc_auc_ovo is (mean : 0.998764) and (standard deviation: 0.000982)
